![alt text](../../figs/logoLEFT.png "CLDF") | CÂMARA LEGISLATIVA DO DISTRITO FEDERAL | ![alt text](../../figs/logoRIGHT.png "CLDF")
--- | :---: | ---
| **COORDENADORIA DE MODERNIZAÇÃO E INFORMÁTICA - CMI** | 
| **OFICINA ASI/LABHINOVA** | 
| **ANÁLISE EXPLORATÓRIA DE DADOS ABERTOS USANDO PYTHON** | 

# Plotando e Visualizando Gráficos Usando Bokeh

#### Cada jupyter-notebook possui "sua própria memória" assim objetos definidos em um não são acessíveis em outro. Ou seja, se você tentar acessar nosso "DataFrame" definido no notebook anterior, o notebook atual não vai reconhecer. Olha só:

In [1]:
df

NameError: name 'df' is not defined

#### Então precisamos carregar tudo de novo para a memória. Sorte que já adiantamos isso para você na próxima linha que importa a biblioteca pandas, lê o arquivo csv e ainda cria as colunas "divisao" e "divisaoMaiorQue180" criadas na etapa anterior da oficina:

In [62]:
import pandas as pd
df = pd.read_csv('bolsafamilia.csv')
df['divisao'] = df['valor'] / df['qtdBeneficiados']
df['divisaoMaiorQue180'] = df['divisao'] > 180
df['data'] = pd.to_datetime(df['dataReferencia'])
df.head()

,id,dataReferencia,codIBGE,nomeIBGE,UFSigla,UFNome,valor,qtdBeneficiados,divisao,divisaoMaiorQue180,data
0,75908792,01/01/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11450358.0,63239,181.064818,True,2013-01-01
1,72208975,01/02/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11034212.0,61121,180.530620,True,2013-01-02
2,63909551,01/03/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11435012.0,61189,186.880191,True,2013-01-03
3,74709431,01/04/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,10762080.0,59505,180.860096,True,2013-01-04
4,73009570,01/05/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11761262.0,67777,173.528808,False,2013-01-05


#### Ótimo! Agora podemos continuar...

## Visualização é importante porque - Falar disso.

#### Esse conjunto de dados apresenta os gastos do bolsa família em cada mês desde 2013 até 2020. Um exercício interessante seria agregar os valores por ano. Ou seja, quanto foi gasto por ano em Brasília? Dessa forma, antes de plotar um gráfico, precisamos fazer essa agregação.

#### Em primeiro lugar, precisamos criar a coluna que usaremos para realizar a agregação. Quero agregar por ano, então precisamos descobrir uma forma de criar uma coluna só com o ano de cada linha. Podemos fazer isso com o comando abaixo:

In [9]:
df['ANO'] = pd.DatetimeIndex(df['data']).year
df.head()

,id,dataReferencia,codIBGE,nomeIBGE,UFSigla,UFNome,valor,qtdBeneficiados,divisao,divisaoMaiorQue180,data,ANO
0,75908792,01/01/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11450358.0,63239,181.064818,True,2013-01-01,2013
1,72208975,01/02/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11034212.0,61121,180.530620,True,2013-01-02,2013
2,63909551,01/03/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11435012.0,61189,186.880191,True,2013-01-03,2013
3,74709431,01/04/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,10762080.0,59505,180.860096,True,2013-01-04,2013
4,73009570,01/05/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11761262.0,67777,173.528808,False,2013-01-05,2013


#### O comando extrai o ano da coluna "data" e armazena numa nova coluna que nós chamamos de "ANO". Em seguida, vamos criar a agregação desejada usando essa nova coluna:

In [10]:
agregacao = df.groupby(['ANO']).sum()
agregacao

,id,codIBGE,valor,qtdBeneficiados,divisao,divisaoMaiorQue180
ANO,,,,,,
2013,835704105,63601296,136882148.0,783516,2099.336388,4.0
2014,830411489,63601296,147621128.0,856821,2068.623134,3.0
2015,828606920,63601296,149440066.0,879250,2045.117841,0.0
2016,814407491,63601296,150499707.0,946384,1904.631152,0.0
2017,840109052,63601296,151730592.0,950536,1915.791988,0.0
2018,838704651,63601296,153313408.0,915218,2013.205879,0.0
2019,809106117,63601296,150125272.0,794778,2268.250440,12.0
2020,67007738,5300108,12109549.0,61776,196.023520,1.0


#### O comando de agregação é o "groupby". Precisamos informar o(s) campo(s) que serão usados para realizar a agregação. Ele devolve um novo DataFrame que nós armazenamos na nova variável "agregacao".  Em seguida, informamos o tipo de operação que deve ser realizado, nesse caso o comando "sum()" indica que queremos somar os valores.

Aqui é importante fazer alguns comentários: Perceba que ele agregou por "ANO" somando os valores. Ele não quis nem saber, trouxe a soma do id, codIBGE, valor, qtdBeneficiados e divisao. 

Aqui o Cientista de Dados deve ter o bom senso de saber que somar "id" ou "codIBGE" que posuem mesmo "ANO" não faz sentido. Agora, somar o "valor" e a "qtdBeneficados" já faz mais sentido. Essa é a impostância de entender o Domínio da Aplicação e entender o que os dados significam.


#### Agora temos um DataFrame chamado "agregacao" onde temos para cada ano, um total de valor gasto no Bolsa Familia. Vamos plotar nosso primeiro gráfico com esses dados. Queremos um gráfico de barras mostrando a progressão dos anos e o total do valor gasto.

#### Para plotar esse gráfico, escolhemos usar a biblioteca Bokeh apresentada anteriormente nos Slides. Então vamos lá:

![alt text](../../figs/bokeh.png "Biblioteca Bokeh")

#### Antes de mais nada vamos pedir ao Bokeh para que os gráficos que iremos plotar sejam exibidos no próprio jupyter-notebook:

In [64]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

#### E agora vamos plotar o gráfico. Dessa vez não vou explicar o código não. Confia e executa:

In [66]:
from bokeh.models import ColumnDataSource
from bokeh.io import show
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import HoverTool

source = ColumnDataSource(dict(x=list(agregacao.index),top=list(agregacao['valor']),))
TOOLS = "reset,crosshair,pan,wheel_zoom,box_zoom"
hover = HoverTool(
    tooltips=[
        ("Indice", "$index"),
        ("Ano", "@x"),
        ("Valor", "R$ @top{0,000,000.0}"),
    ],
    formatters={
        'Ano': 'printf',
        'Valor': 'printf',
    },
    mode='mouse'
)

p = figure(tools=TOOLS, plot_width=700, plot_height=300, title="Gastos com Bolsa Família no DF por ano")
p.grid.grid_line_alpha = 0.3
p.add_tools(hover)

p.vbar(x='x', top='top',bottom=0, width=0.5, fill_color="blue", source=source)

p.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
p.add_layout(Grid(dimension=1, ticker=yaxis.ticker))

xaxis = LinearAxis()
plot.add_layout(xaxis, 'left')

yaxis = LinearAxis()
plot.add_layout(yaxis, 'below')

show(p)



#### Seria importante dizer algo sobre isso...